# **How to measure the runtime & electricity consumption of an ML model ? Few examples using the tool [CodeCarbon](https://codecarbon.io/#howitwork)**

GitHub repo : https://github.com/mlco2/codecarbon
CodeCarbon documentation : https://mlco2.github.io/codecarbon/

#### **Example with a classification model**

### 1. Import modules

In [1]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score 

from codecarbon import EmissionsTracker

from transformers import pipeline
from datasets import Dataset, load_dataset
from transformers import pipeline, AutoTokenizer
from codecarbon import EmissionsTracker

import torch




c:\Users\soury\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Create a DataFrame from preprocessed dataset

For this example, we used an open dataset about Rain in Australia, you can find the complete database [here](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package)

In [2]:
df = pd.read_csv('./preprocessed.csv')
df.head(10)

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,RainToday,RainTomorrow
0,31,14.3,14.3,14.2,3.800000,0.000000,5,39.00000,5,9,17.0,0.0,94.0,1005.00000,1005.000000,8.000000,8.00000,1.0,1.0
1,0,9.6,9.6,8.4,5.468232,6.200000,15,50.00000,7,15,20.0,28.0,58.0,1013.10000,1016.300000,4.447461,4.50993,1.0,0.0
2,5,9.9,9.9,0.0,5.468232,7.611178,11,28.00000,11,11,11.0,11.0,47.0,1012.60000,1012.400000,4.000000,2.00000,0.0,1.0
3,43,12.3,12.3,0.0,5.468232,7.611178,11,37.00000,8,8,15.0,20.0,57.0,1019.80000,1021.000000,4.447461,4.50993,0.0,0.0
4,37,10.1,10.1,0.0,3.000000,9.800000,13,40.03523,13,8,20.0,11.0,32.0,1016.40000,1016.400000,2.000000,1.00000,0.0,0.0
5,39,15.9,15.9,0.0,4.400000,7.611178,4,30.00000,9,1,15.0,20.0,63.0,1016.80000,1013.300000,1.000000,4.50993,0.0,0.0
6,17,8.2,8.2,22.5,5.468232,7.611178,10,39.00000,9,2,6.0,15.0,45.0,1017.64994,1015.255889,4.447461,8.00000,1.0,0.0
7,40,-1.7,-1.7,0.0,5.468232,7.611178,6,20.00000,3,6,0.0,9.0,46.0,1031.30000,1026.400000,4.447461,4.50993,0.0,0.0
8,28,10.4,10.4,0.0,6.400000,11.100000,3,50.00000,5,5,22.0,24.0,29.0,1022.70000,1019.300000,3.000000,6.00000,0.0,0.0
9,47,18.2,18.2,0.0,5.468232,7.611178,4,44.00000,3,4,17.0,35.0,67.0,1021.00000,1015.300000,4.447461,4.50993,0.0,0.0


In [3]:
df.shape

(13665, 19)

#### 2.1 Shape of DataFrame

In [4]:
print(f'Number of row : {df.shape[0] }')
print(f'Number of columns : {df.shape[1]}')

Number of row : 13665
Number of columns : 19


#### 2.2 Describing the attributes

In [5]:
#if it is well preprocessed, you should only have int & float types (numbers)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13665 entries, 0 to 13664
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       13665 non-null  int64  
 1   MinTemp        13665 non-null  float64
 2   MaxTemp        13665 non-null  float64
 3   Rainfall       13665 non-null  float64
 4   Evaporation    13665 non-null  float64
 5   Sunshine       13665 non-null  float64
 6   WindGustDir    13665 non-null  int64  
 7   WindGustSpeed  13665 non-null  float64
 8   WindDir9am     13665 non-null  int64  
 9   WindDir3pm     13665 non-null  int64  
 10  WindSpeed9am   13665 non-null  float64
 11  WindSpeed3pm   13665 non-null  float64
 12  Humidity3pm    13665 non-null  float64
 13  Pressure9am    13665 non-null  float64
 14  Pressure3pm    13665 non-null  float64
 15  Cloud9am       13665 non-null  float64
 16  Cloud3pm       13665 non-null  float64
 17  RainToday      13665 non-null  float64
 18  RainTo

#### 3. Split the dataset into train & test dataset

In [6]:
# only the last column
X=df.iloc[:,:-1]
# all the columns except the last one 
y=df.iloc[:,-1]

In [7]:
# we split the dataset in 2 parts, 80% of training data and 20% of testing data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
print(f'The number of features of the training dataset is {x_train.shape[1] } and the number of lines is {x_train.shape[0]}')
print(f'The number of features of the test dataset is {x_test.shape[1] } and the number of lines is {x_test.shape[0]}')

The number of features of the training dataset is 18 and the number of lines is 10932
The number of features of the test dataset is 18 and the number of lines is 2733


#### 4. Train a model and make measurements

#### 4.1 A basic example

In [10]:
#Simple example to measure the training of a classification model named KNeighborsClassifier
classifier= KNeighborsClassifier()

#track the ml model training
# /!\ the trianing is on 10932 lines   
tracker = EmissionsTracker(project_name="example1_train_"+str(x_train.shape[1])+"features_"+str(x_train.shape[0])+"lines")

tracker.start()

classifier.fit(x_train, y_train)

#stop the tracker at the end of the training
tracker.stop()

#track the model inference 
# /!\ the inference is for 5 lines
tracker = EmissionsTracker(project_name="example1_inference_"+str(x_test.shape[1])+"features_5lines")
tracker.start()
classifier.predict(x_test.iloc[0:5])
emissions: float = tracker.stop()
print(emissions)

#when the run is done, the results are save in a new line in the emission.csv file 
#(the file is created if it doesn't already exist)

[codecarbon INFO @ 16:21:01] [setup] RAM Tracking...
[codecarbon INFO @ 16:21:01] [setup] GPU Tracking...
[codecarbon INFO @ 16:21:01] No GPU found.
[codecarbon INFO @ 16:21:01] [setup] CPU Tracking...
[codecarbon DEBUG @ 16:21:01] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : expected str, bytes or os.PathLike object, not NoneType
[codecarbon DEBUG @ 16:21:01] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Platform not supported by Intel RAPL Interface
[codecarbon DEBUG @ 16:21:01] Not using PowerMetrics, an exception occurred while instantiating Powermetrics : Platform not supported by Powermetrics
[codecarbon WARNING @ 16:21:01] No CPU tracking mode found. Falling back on CPU constant mode.


[codecarbon DEBUG @ 16:21:03] CPU : We detect a 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz with a TDP of 28 W
[codecarbon INFO @ 16:21:03] CPU Model on constant consumption mode: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz
[codecarbon INFO @ 16:21:03] >>> Tracker's metadata:
[codecarbon INFO @ 16:21:03]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 16:21:03]   Python version: 3.12.1
[codecarbon INFO @ 16:21:03]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 16:21:03]   Available RAM : 15.688 GB
[codecarbon INFO @ 16:21:03]   CPU count: 8
[codecarbon INFO @ 16:21:03]   CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz
[codecarbon INFO @ 16:21:03]   GPU count: None
[codecarbon INFO @ 16:21:03]   GPU model: None
[codecarbon DEBUG @ 16:21:03] Not running on AWS
[codecarbon DEBUG @ 16:21:03] Not running on Azure
[codecarbon DEBUG @ 16:21:03] Not running on GCP
[codecarbon INFO @ 16:21:04] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.1494355201721191

None


#### 4.2 Perform loop measurements on different algorithms

**Basic classification algorithms**

In [11]:
# you can change the list with any other algorithm
classifiers = {
    KNeighborsClassifier():0,
    GaussianNB():0,
    RandomForestClassifier():0,
}

In [12]:
model_dict = {}

for classifier in classifiers:


    #track the ml model training    
    # /!\ the training is on 10932 lines   
    tracker = EmissionsTracker(project_name=str(classifier)+"_train_"+str(x_train.shape[1])+"features_"+str(x_train.shape[0])+"lines")
    tracker.start()
    classifier.fit(x_train, y_train)
    tracker.stop()

    #track the ml model use
    # /!\ the inference is for 2733 lines
    tracker = EmissionsTracker(project_name=str(classifier)+"_inference_"+str(x_test.shape[1])+"features_"+str(x_test.shape[0])+"lines")
    tracker.start()
    predicted=classifier.predict(x_test)
    _=tracker.stop()

    #save the accuracy
    model_dict[classifier] = accuracy_score(y_test, predicted)*100  
    print ("The accuracy of the classifier is : ", accuracy_score(y_test, predicted)*100, "%")


[codecarbon INFO @ 16:21:12] [setup] RAM Tracking...
[codecarbon INFO @ 16:21:12] [setup] GPU Tracking...
[codecarbon INFO @ 16:21:12] No GPU found.
[codecarbon INFO @ 16:21:12] [setup] CPU Tracking...
[codecarbon DEBUG @ 16:21:12] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : expected str, bytes or os.PathLike object, not NoneType
[codecarbon DEBUG @ 16:21:12] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Platform not supported by Intel RAPL Interface
[codecarbon DEBUG @ 16:21:12] Not using PowerMetrics, an exception occurred while instantiating Powermetrics : Platform not supported by Powermetrics
[codecarbon WARNING @ 16:21:12] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon DEBUG @ 16:21:14] CPU : We detect a 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz with a TDP of 28 W
[codecarbon INFO @ 16:21:14] CPU Model on constant consumption mode: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 

The accuracy of the classifier is :  84.30296377607024 %


[codecarbon DEBUG @ 16:21:19] CPU : We detect a 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz with a TDP of 28 W
[codecarbon INFO @ 16:21:19] CPU Model on constant consumption mode: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz
[codecarbon INFO @ 16:21:19] >>> Tracker's metadata:
[codecarbon INFO @ 16:21:19]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 16:21:19]   Python version: 3.12.1
[codecarbon INFO @ 16:21:19]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 16:21:19]   Available RAM : 15.688 GB
[codecarbon INFO @ 16:21:19]   CPU count: 8
[codecarbon INFO @ 16:21:19]   CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz
[codecarbon INFO @ 16:21:19]   GPU count: None
[codecarbon INFO @ 16:21:19]   GPU model: None
[codecarbon DEBUG @ 16:21:19] Not running on AWS
[codecarbon DEBUG @ 16:21:19] Not running on Azure
[codecarbon DEBUG @ 16:21:19] Not running on GCP
[codecarbon INFO @ 16:21:20] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.1513881683349609

The accuracy of the classifier is :  80.20490303695573 %


[codecarbon DEBUG @ 16:21:24] CPU : We detect a 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz with a TDP of 28 W
[codecarbon INFO @ 16:21:24] CPU Model on constant consumption mode: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz
[codecarbon INFO @ 16:21:24] >>> Tracker's metadata:
[codecarbon INFO @ 16:21:24]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 16:21:24]   Python version: 3.12.1
[codecarbon INFO @ 16:21:24]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 16:21:24]   Available RAM : 15.688 GB
[codecarbon INFO @ 16:21:24]   CPU count: 8
[codecarbon INFO @ 16:21:24]   CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz
[codecarbon INFO @ 16:21:24]   GPU count: None
[codecarbon INFO @ 16:21:24]   GPU model: None
[codecarbon DEBUG @ 16:21:24] Not running on AWS
[codecarbon DEBUG @ 16:21:24] Not running on Azure
[codecarbon DEBUG @ 16:21:24] Not running on GCP
[codecarbon INFO @ 16:21:26] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.1540060043334961

The accuracy of the classifier is :  85.32747896084888 %


**Print the accuracy of each model**

In [13]:
for cle, valeur in model_dict.items():
  print(cle, " -> Accuracy : ", valeur)

KNeighborsClassifier()  -> Accuracy :  84.30296377607024
GaussianNB()  -> Accuracy :  80.20490303695573
RandomForestClassifier()  -> Accuracy :  85.32747896084888


**If you run this script, you obtain a dataset of 8 lines, one for each of the tracker.start/tracker.stop**
**With the file obtained, you can get many informations like the electricty consumed by the CPU (kWh), the RAM & the GPU, the CPU model, the cloud provider region ...**

#### **Example with a Langage Model**

#### Example for fine tuning 

The script was inspired by : https://www.datacamp.com/tutorial/fine-tuning-llama-2

In [ ]:

# Dataset
data_name = "mlabonne/guanaco-llama2-1k"
training_data = load_dataset(data_name, split="train")
# Fine-tuned model
new_model = "llama-2-7b-chat-guanaco"

# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "llama-2-7b-mlabonne-enhanced" #You can give it your own name

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

## Measurements with codecarbon 

tracker = EmissionsTracker(project_name="Llama_7b_finetuning")
tracker.start()

# Trainer
trainer = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
trainer.train()

tracker.stop()

trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)



#### Example for inference

In [ ]:
prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=llama_tokenizer, max_length=200)

#track the consumption related to the generation of 1 answer with the fintuned model
tracker = EmissionsTracker(project_name="Llama_7b_inference")
tracker.start()
result = pipe(f"<s>[INST] {prompt} [/INST]")
tracker.stop()

print(result[0]['generated_text'])


You can find further examples here : 
https://github.com/mlco2/codecarbon/blob/master/examples & 
https://github.com/sashavor/co2_inference/tree/main/code